In [13]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-0rLvuRkMiD4Mw25QYygh6rUlZVjpQWNGNF4yez7z3PZ7yCOm",
    # api_key="sk-tMbkq3K1iO5vf0FRMlrmzslGXJZwE0us3mve4QXuvpnZcumG",
    base_url="https://api.chatanywhere.tech/v1"
    # base_url="https://api.chatanywhere.cn/v1"
)

In [14]:
def gpt_35_api(messages: list):

    completion = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=messages, temperature=0.5)
    return completion.choices[0].message.content

In [15]:
import json
from tqdm import tqdm

In [16]:
from langchain import hub

prompt = hub.pull("tyfann/llm4commit-rag")

In [17]:
with open('../data/chronicle/data_with_similar_diff.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

In [18]:
gpt_msg = []
for data in tqdm(org_data[:100], total=len(org_data[:100]), desc="Processing documents"):
    messages = prompt.invoke(
        {"context": data['sim_diff'] + "\nCommit message:"+ data['sim_msg'], "question": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    gpt_msg.append(gpt_35_api(example_prompt))

Processing documents: 100%|██████████| 100/100 [02:29<00:00,  1.50s/it]


In [19]:
with open('../data/chronicle/chronicle_chatgpt.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)

for item, msg in zip(chronicle_data, gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/chronicle/chronicle_chatgpt_rag.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data, f, ensure_ascii=False, indent=4)